In [1]:
import numpy as np
import sys
import hrr_common.utils as cu
import hrr_common as hrr_cm
import hrr_cobot_robot as hrr_rob
import spatialmath as sm
import rospy
from tqdm.notebook import tqdm, trange
from geometry_msgs.msg import Vector3Stamped, PointStamped
from hrr_msgs.msg import PlannerAction , PlannerGoal
from hrr_cobot_robot.gripper_interfaces.wsg_50_hw_local import gripper_command_handle as gch
# ROS
import actionlib

### ros setup for the lab

In [2]:
import socket
print("for a local setup, replace the above by:\n\t" + \
      r"%env ROS_MASTER_URI=http://localhost:11311")
%env ROS_MASTER_URI=http://hrrN3511rt2004.lsr.ei.tum.de:11311
print("in case this notebook does not work please check that IP below is identical:\n\t"+
      r"%env " + f"ROS_IP={ socket.gethostbyname(socket. gethostname())}\t\t<=>",end="\t")
%env ROS_IP=129.187.147.174

for a local setup, replace the above by:
	%env ROS_MASTER_URI=http://localhost:11311
env: ROS_MASTER_URI=http://hrrN3511rt2004.lsr.ei.tum.de:11311
in case this notebook does not work please check that IP below is identical:
	%env ROS_IP=129.187.147.174		<=>	env: ROS_IP=129.187.147.174


In [3]:
rospy.init_node('pushing_dev')

# Actual grasping routine

### Use case: Push PSU from the outside

### Init robot control

In [5]:
hrr_rob.load_default_parameters("/hrr_cobot")
cobot = hrr_rob.HrrCobotControl.from_ros()
rospy.set_param("~joint_state_topic_name", "/hrr_cobot/joint_states")
rospy.set_param("~tp5_topic_name", "/hrr_cobot/comau_robot_state_controller/comau_cartesian_data")
comau_status = hrr_rob.ComauRobotState.from_ros()
rospy.sleep(1e-1)
print(comau_status)

[ERROR] [1642080643.879879]: could not set tool-frame @ /pushing_dev due to missing ROS-parameter: '~tool_name'


[INFO] [1642080644.017694]: Updated force  gains: K_p:=[0.0001, 0.0001, 0.0001]
[INFO] [1642080644.019583]: Updated torque gains: K_p:=[0.0005, 0.0005, 0.0005]
q:		[-7.07486135e-02  6.40425417e-02 -9.00067234e+01  6.64071345e-02
  8.99218651e+01 -1.25206332e-01][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[4.36425110e-01 4.46169287e-04 7.04523010e-01][m]
quat:		0.00005 <[ 4.74548848e-04  9.99999768e-01 -4.85068907e-04]>
tool-pos:	[4.36425110e-01 4.46169287e-04 7.04523010e-01][m]
tool-rpy:	[-1.79945618e+02  6.23249868e-03  1.79944412e+02][°]
robot-status:	ready


### Parameters given from outside

#Normal vector on the surface of which to push (i.e. the opposite of the direction we want to push in)
surface_normal = PointStamped()#np.array([0, 1, 0]) #pointStamped
#location
#where we want to start pushing (for testing use current pose as goal_pose)
contact_point = Vector3Stamped() #vector3stamped 
#pushing distance
push_dist = 5e-2 #in meters

### Compute goal pose

nvec = np.array([surface_normal.point.x, surface_normal.point.y, surface_normal.point.z])
location = np.array([contact_point.vector.x, contact_point.vector.y, contact_point.vector.z])
C_p_CE = np.array([0,0,0]) #displacement from tip of tool to last link in tip of tool frame
goal_pose = cu.calc_EE_goal_pose(nvec,location,C_p_CE)

In [8]:
#Using Gripper
gripper = gch.GripperCommandHandle('/hrr_cobot/gripper')
gripper.reset()
#Also, orientation needs to be done such that the fingers point in -nvec direction

In [7]:
gripper.send_pos(20,100) #closes gripper to 20mm with 100 mm/s closing speed
#We assume that we push with the fingers of the gripper, while the fingers are closed
#We assume the closed fingers are directly beneath the EE of the robot, 
#i.e. the transformation from EE to tip of fingers is only a translation
#Now location needs to be translated by that distance between tip of fingers and robot EE

### Alternatively, use manual goal pose and nvec (for demo etc)

In [12]:
#np.save('goal_pose_pushing',cobot.T_B_E_robot.A,allow_pickle=True) #saves current pose as goal_pose
goal_pose = np.load('goal_pose_pushing.npy')
nvec = np.array([0, 1, 0]) #surface normal
sm.SE3(goal_pose)

  -0.6646   -0.7472   -9.467e-05  0.6477    
   0.003176 -0.002698 -1         0.07612   
   0.7472   -0.6646    0.004166  0.09073   
   0         0         0         1         


### Compute pre-pose and end-of-push-pose

In [31]:
#desired distance from goal-pose to pre-pose
planner_pre_pose_dist = 0.05 #m
pre_pose_dist = 0.05 #m
push_dist = 0.03 #m

In [32]:
#Compute pre-pose from nvec, location, pre_pose_dist, disp_tip_EE
nvec = cu.normalize_vec(nvec) #Make sure nvec is normalized
pre_pose = sm.SE3(nvec* pre_pose_dist) @ sm.SE3(goal_pose)
pre_pose

  -0.6646   -0.7472   -9.467e-05  0.6477    
   0.003176 -0.002698 -1         0.1261    
   0.7472   -0.6646    0.004166  0.09073   
   0         0         0         1         


In [33]:
endofpush_pose = sm.SE3(-nvec* push_dist) @ sm.SE3(goal_pose)
endofpush_pose

  -0.6646   -0.7472   -9.467e-05  0.6477    
   0.003176 -0.002698 -1         0.04612   
   0.7472   -0.6646    0.004166  0.09073   
   0         0         0         1         


In [34]:
planner_prepose = sm.SE3(np.array([0,0,1])*planner_pre_pose_dist) @ sm.SE3(pre_pose)
planner_prepose

  -0.6646   -0.7472   -9.467e-05  0.6477    
   0.003176 -0.002698 -1         0.1261    
   0.7472   -0.6646    0.004166  0.1407    
   0         0         0         1         


### Init planner

In [ ]:
 # Initialize the planner action client.
    
# >> Remeber to run the planner node (rosrun planner planner_server_v2.py) in terminal
# >> 
#rospy.init_node('planner_client')

client1 = actionlib.SimpleActionClient('planner', PlannerAction)
client1.wait_for_server()

# Sending deride end configuration
goal = PlannerGoal()
goal.goal_pose.header.frame_id = "map"
goal.goal_pose.header.stamp = rospy.Time.now()

### Start at calibration pose

cobot.move_to_pose(cobot.FK(cobot.q_calib), err_gain = None, v_max = 0.03) #start_pose, cobot.FK(cobot.q_calib)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

In [36]:
pre_pose_quat = cu.homog2quat(cobot.FK(cobot.q_calib))
pre_pose_pos = cu.homog2pos(cobot.FK(cobot.q_calib))
#position of end_configuration
goal.goal_pose.pose.position.x = pre_pose_pos[0]
goal.goal_pose.pose.position.y = pre_pose_pos[1]
goal.goal_pose.pose.position.z = pre_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = pre_pose_quat.x
goal.goal_pose.pose.orientation.y = pre_pose_quat.y
goal.goal_pose.pose.orientation.z = pre_pose_quat.z
goal.goal_pose.pose.orientation.w = pre_pose_quat.w
client1.send_goal(goal)

### Go to pre-pose using traj planner

In [37]:
pre_pose_quat = cu.homog2quat(planner_prepose)
pre_pose_pos = cu.homog2pos(planner_prepose)
#position of end_configuration
goal.goal_pose.pose.position.x = pre_pose_pos[0]
goal.goal_pose.pose.position.y = pre_pose_pos[1]
goal.goal_pose.pose.position.z = pre_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = pre_pose_quat.x
goal.goal_pose.pose.orientation.y = pre_pose_quat.y
goal.goal_pose.pose.orientation.z = pre_pose_quat.z
goal.goal_pose.pose.orientation.w = pre_pose_quat.w

In [38]:
client1.send_goal(goal)
gripper.send_pos(20,50)

### Approach and push

In [39]:
cobot.move_to_pose(pre_pose)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1642081583.422478]: reached goal pose at step 195 / 10000


In [40]:
cobot.move_to_pose(endofpush_pose)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1642081585.758239]: reached goal pose at step 228 / 10000


### Go back to pre-pose

In [41]:
cobot.move_to_pose(pre_pose)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1642081588.596282]: reached goal pose at step 226 / 10000


In [42]:
cobot.move_to_pose(planner_prepose)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()


  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1642081590.512234]: reached goal pose at step 186 / 10000


### Go back to calib pose

In [43]:
gripper.reset()

pre_pose_quat = cu.homog2quat(cobot.FK(cobot.q_calib))
pre_pose_pos = cu.homog2pos(cobot.FK(cobot.q_calib))
#position of end_configuration
goal.goal_pose.pose.position.x = pre_pose_pos[0]
goal.goal_pose.pose.position.y = pre_pose_pos[1]
goal.goal_pose.pose.position.z = pre_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = pre_pose_quat.x
goal.goal_pose.pose.orientation.y = pre_pose_quat.y
goal.goal_pose.pose.orientation.z = pre_pose_quat.z
goal.goal_pose.pose.orientation.w = pre_pose_quat.w
client1.send_goal(goal)

[WARN] [1642081693.671409]: Could not process inbound connection: Client [/hrr_cobot] wants topic [/hrr_cobot/sensor_track_compliant_controller/hybrid_ctrl_cmd] to have datatype/md5sum [hrr_msgs/HybridForceVelocityCmdStamped/6bfe9d92f9569269ce7086e736ecd0c2], but our version has [hrr_msgs/HybridForceVelocityCmdStamped/c7d32c882fe005053aba472f934842bd] Dropping connection.{'callerid': '/hrr_cobot', 'md5sum': '6bfe9d92f9569269ce7086e736ecd0c2', 'tcp_nodelay': '0', 'topic': '/hrr_cobot/sensor_track_compliant_controller/hybrid_ctrl_cmd', 'type': 'hrr_msgs/HybridForceVelocityCmdStamped'}
